In [11]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [13]:
import math
import pickle
from tqdm import tqdm_notebook as tqdm

In [14]:
# import statements for selenium

from selenium import webdriver

options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('/home/tensor/Documents/softwares/chromedriver', chrome_options=options)

/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [15]:
main_url = "https://www.zalando.co.uk/womens-clothing/"
base_url = "https://www.zalando.co.uk/"
base_path = "/media/tensor/EXTDRIVE/projects/virtual-try-on/dataset/zalando_data/pickles/"

In [5]:
# getting the list of all the categories along with sub-categories
html = urlopen(main_url)
soup = BeautifulSoup(html, 'lxml')

# getting the main categories
main_categories = []
anchors = soup.findAll("a", {"class": "cat_link-mTK6o"})
for anchor in anchors:
    main_categories.append(anchor.get("href").split("/")[1])
    
main_categories = main_categories[1:]
print("main categories")
print("________________________________________________")
print(main_categories)
print("------------------------------------------------")

# getting the sub categories
categories = {} # dictionary to store all the sub categories along with main categories
for category in main_categories:
    url = base_url + category +"/"
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    anchors = soup.findAll(True, {"class": "cat_link-mTK6o cat_highlight-2DmxP"})[0].find_next('ul').findAll("a", {"class": "cat_link-mTK6o"})
    category_list = []
    for anchor in anchors:
        category_list.append(anchor.get("href").split("/")[1])
    categories[category] = category_list

print("categories with sub categories")
print("_________________________________________________")
print(categories)
print("-------------------------------------------------")

main categories
________________________________________________
['womens-clothing-blouses-tunics', 'womens-clothing-coats', 'womens-clothing-dresses', 'womens-clothing-jackets', 'womens-clothing-jeans', 'womens-clothing-jumpers-cardigans', 'womens-clothing-lingerie-nightwear', 'womens-clothing-skirts', 'womens-sports-clothing', 'womens-clothing-swimwear', 'womens-clothing-tights-socks', 'womens-clothing-tops', 'womens-clothing-trousers-leggings']
------------------------------------------------
categories with sub categories
_________________________________________________
{'womens-clothing-blouses-tunics': ['womens-clothing-blouses', 'womens-clothing-shirts', 'womens-clothing-tunics'], 'womens-clothing-coats': ['womens-clothing-coats-down-coats', 'womens-clothing-coats-parkas', 'womens-clothing-coats-short-coats', 'womens-clothing-coats-trench-coats', 'womens-clothing-coats-winter-coats', 'womens-clothing-coats-wool-coats'], 'womens-clothing-dresses': ['summer-dresses', 'cocktail-dr

In [6]:
# # saving the categories information in a pkl file
# with open("./pickles/categories.pkl", "wb") as f:
#     pickle.dump(categories, f)

In [16]:
# loading the categories information from the pickle file
with open("./pickles/categories.pkl", "rb") as f:
    categories = pickle.load(f)

In [17]:
categories_copy = categories.copy()

In [16]:
# Going through all the sub categories and saving them as a pickle file containing only the links

for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    
    if not os.path.exists(main_category_path):
        os.makedirs(main_category_path)
        
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
        
        if not os.path.exists(sub_category_path):
            os.makedirs(sub_category_path)
            
        url = base_url + sub_category
        
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        
        # now getting the total number of pages for each category from total items, taking 84 items for each page
        total_items = int(soup.find("h3", {"class": "cat_count-1RzWf"}).get_text().split(" ")[0].replace(',', ''))
        total_pages = math.ceil(total_items/84)
        
        # now opening each of the pages in selenium because in beautiful soup all the images are not available
        # and putting all the urls to as a list in pickle format
        items = []
        for i in tqdm(range(total_pages)):
            driver.get(url + "?p=" + str(i))
            contents = driver.find_elements_by_class_name('cat_imageLink-OPGGa')
            for content in contents:
                items.append(content.get_attribute('href'))
        
        with open(sub_category_path + "/" + sub_category + ".pkl", "wb") as f:
            pickle.dump(items, f)

/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [18]:
categories_copy = categories.copy()

In [9]:
# going through each of the subcategories and saving their each image urls according to the try-on format
for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
        # opening pickle file containing the links
        with open(sub_category_path + "/" + sub_category +".pkl", "rb") as f:
            items = pickle.load(f)
        file_path = sub_category_path + "/" + sub_category + "_try-on.pkl"
        with open(file_path, "rb") as f:
            dummy = pickle.load(f)
        if len(list(dummy.keys())) == 0:
            try_on_dict = {}
            for item in tqdm(items):
                try:
                    url = item
                    html = urlopen(url)
                    soup = BeautifulSoup(html, "lxml")

                    images = soup.find("div", {"class": "_8Nfi4s _2x18vI"}).find_all("img")
                    if len(images)>4:
                        product_images = []
                        normal_images = []
                        for img in images:
                            link = img['src']
                            if "packshot" in link:
                                product_images.append(link.split("?")[0])
                            else:
                                normal_images.append(link.split("?")[0])
                        try_on = []
                        try_on.append(product_images[0])
                        try_on.append(normal_images[0])
                        try_on.append(normal_images[1])
                        try_on_dict[url] = try_on
                except:
                    pass

            # saving the description in pickle format
            with open(sub_category_path + "/" + sub_category + "_try-on.pkl", "wb") as f:
                pickle.dump(try_on_dict, f)                    

/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
